# Importing Libraries

In [ ]:
pip install pandas dask duckdb xgboost scikit-learn shap matplotlib seaborn boto3

In [1]:
!ls

sample_data


In [ ]:
# Install the necessary Dask package
!pip install dask[dataframe] --quiet

# Import necessary libraries
import dask.dataframe as dd
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import shap
import boto3
import json
import os

# Function to recommend retention strategies
def recommend_action(churn_probability):
    """Recommend actions based on churn probability."""
    if churn_probability > 0.75:
        return "High Risk: Offer a discount or special service."
    elif 0.50 < churn_probability <= 0.75:
        return "Moderate Risk: Send a personalized email to re-engage."
    else:
        return "Low Risk: No immediate action needed."



In [ ]:
!pwd

/content


In [ ]:
# Import necessary libraries
from google.colab import files
import dask.dataframe as dd
import duckdb
import glob

# Step 1: Upload files from local machine
uploaded_files = files.upload()  # This will prompt you to select files to upload


In [ ]:
import dask.dataframe as dd


In [ ]:
import dask.dataframe as dd
import duckdb
import glob

# Load all Parquet files using Dask
parquet_files = glob.glob("train_*.parquet")  # Adjust the path as needed
print(f"Total Parquet files loaded: {len(parquet_files)}")  # Check the number of files loaded
dask_df = dd.read_parquet(parquet_files)

# Create the transaction amount feature as a proxy
dask_df['transaction_amount'] = (
    dask_df['bank_transfer_in_volume'] +
    dask_df['bank_transfer_out_volume'] +
    dask_df['crypto_in_volume'] +
    dask_df['crypto_out_volume']
)

# Convert Dask DataFrame to a Pandas DataFrame
# Note: Use `.compute()` only if the data fits in memory
pandas_df = dask_df.compute()


# Connect to DuckDB
con = duckdb.connect()

# Create a DuckDB table from the Pandas DataFrame for SQL querying
con.execute("CREATE TABLE bank_data AS SELECT * FROM pandas_df")

# Check unique values in the churn column
churn_counts = pandas_df['churn_due_to_fraud'].value_counts()  # Adjust 'churn_due_to_fraud' if needed
print("Churn Counts:\n", churn_counts)  # Print churn counts

In [ ]:
len(dask_df)

In [ ]:
con.execute("DESCRIBE bank_data").fetchdf()

#**Step 2: Data Exploration and Time-Series Analysis**

In [ ]:
import matplotlib.pyplot as plt

# Ensure the SQL query uses the correct column names
# Use the correct date column name in place of 'date'
time_series_data = con.execute("""
    SELECT
        YEAR(date) AS year,
        SUM(transaction_amount) AS total_transaction
    FROM bank_data
    GROUP BY year
    ORDER BY year
""").fetchdf()

# Check if time_series_data is empty
if time_series_data.empty:
    print("No data found for the time series analysis.")
else:
    # Plotting the time series with enhanced visualization
    plt.figure(figsize=(12, 6))
    plt.plot(time_series_data['year'], time_series_data['total_transaction'], marker='o', color='b')
    plt.title('Total Transaction Amount Over Years', fontsize=16)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Total Transaction Amount', fontsize=14)
    plt.xticks(time_series_data['year'], rotation=45)
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
# Run a SQL query to summarize transaction amounts by year
time_series_data = con.execute("""
    SELECT
        YEAR(date) AS year,
        SUM(bank_transfer_out_volume) AS total_transaction
    FROM bank_data
    GROUP BY year
    ORDER BY year
""").fetchdf()

# Plotting the time series with enhanced visualization
plt.figure(figsize=(12, 6))
plt.plot(time_series_data['year'], time_series_data['total_transaction'], marker='o', color='b')
plt.title('Total Transaction Amount Over Years', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Total Transaction Amount', fontsize=14)
plt.xticks(time_series_data['year'], rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
con.execute("DESCRIBE bank_data").fetchdf()


# **Step 3: Feature Engineering**

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Load and inspect your data
# Assuming `dask_df` is already loaded

# Convert 'date' to datetime
dask_df['date'] = dd.to_datetime(dask_df['date'])

# Calculate recency
current_date = pd.Timestamp.now()
dask_df['recency'] = (current_date - dask_df['date']).dt.days


# Group by 'customer_id' and aggregate
features = dask_df.groupby('customer_id').agg({
    'recency': 'min',  # Minimum recency
    'transaction_amount': 'mean',  # Average transaction amount
    'churn_due_to_fraud': 'max'  # Assuming this indicates churn
}).reset_index()

# Compute the result
features = features.compute()

# Preview the features
print(features.head())

In [ ]:
dask_df.head()

# **Step 4: Data Visualization for Feature Insights**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have your DataFrame named pandas_df
# Calculate skewness for each column
skewness = pandas_df.skew().sort_values(ascending=False)

# Create a DataFrame for better visualization
skewness_df = pd.DataFrame({'Feature': skewness.index, 'Skewness': skewness.values})

# Set the figure size
plt.figure(figsize=(12, 6))

# Plot the skewness
sns.barplot(x='Skewness', y='Feature', data=skewness_df, palette='viridis')
plt.axvline(0, color='red', linestyle='--')  # Add a line at x=0 for reference
plt.title('Skewness of Features', fontsize=16)
plt.xlabel('Skewness', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have your DataFrame named pandas_df
# Select only numeric columns
numeric_df = pandas_df.select_dtypes(include=['number'])

# Calculate skewness for each numeric column
skewness = numeric_df.skew().sort_values(ascending=False)

# Create a DataFrame for better visualization
skewness_df = pd.DataFrame({'Feature': skewness.index, 'Skewness': skewness.values})

# Set the figure size
plt.figure(figsize=(12, 6))

# Plot the skewness
sns.barplot(x='Skewness', y='Feature', data=skewness_df, palette='viridis')
plt.axvline(0, color='red', linestyle='--')  # Add a line at x=0 for reference
plt.title('Skewness of Numeric Features', fontsize=16)
plt.xlabel('Skewness', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have your DataFrame named pandas_df
# Select only numeric columns
numeric_df = pandas_df.select_dtypes(include=['number'])

# Calculate skewness for each numeric column
skewness = numeric_df.skew()

# Set the figure size for the plots
plt.figure(figsize=(15, 5 * len(skewness)))

# Iterate through each numeric column and plot skewness
for i, column in enumerate(numeric_df.columns):
    plt.subplot(len(numeric_df.columns), 1, i + 1)  # Create a subplot for each feature
    sns.histplot(numeric_df[column], kde=True, color='skyblue', bins=30)
    plt.axvline(numeric_df[column].mean(), color='red', linestyle='--', label='Mean')
    plt.axvline(numeric_df[column].median(), color='green', linestyle='--', label='Median')
    plt.title(f'Distribution of {column} (Skewness: {skewness[column]:.2f})', fontsize=16)
    plt.xlabel(column, fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.legend()
    plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
# Make sure features DataFrame has the correct columns
# Check the columns in the features DataFrame
print(features.columns)

# Visualizing the distribution of churn
plt.figure(figsize=(8, 5))
sns.countplot(data=features, x='churn_due_to_fraud', palette='pastel')
plt.title('Churn Distribution', fontsize=16)
plt.xlabel('Churn', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks([0, 1], ['Not Churned', 'Churned'], rotation=0)
plt.grid()
plt.tight_layout()
plt.show()

# Visualizing the average transaction amount by churn status
plt.figure(figsize=(10, 6))
sns.barplot(data=features, x='churn_due_to_fraud', y='transaction_amount', palette='viridis')
plt.title('Average Transaction Amount by Churn Status', fontsize=16)
plt.xlabel('Churn', fontsize=14)
plt.ylabel('Average Transaction Amount', fontsize=14)
plt.xticks([0, 1], ['Not Churned', 'Churned'], rotation=0)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Pie chart for churn distribution
plt.figure(figsize=(8, 6))
churn_counts = features['churn_due_to_fraud'].value_counts()
plt.pie(churn_counts, labels=['Not Churned', 'Churned'], autopct='%1.3f%%', startangle=90, colors=['#4CAF50', '#FF5722'])
plt.title('Churn Distribution', fontsize=16)
plt.axis('equal')  # Equal aspect ratio ensures that pie chart is circular
plt.tight_layout()
plt.show()

In [ ]:
# Box plot for transaction amount by churn status
plt.figure(figsize=(10, 6))
sns.boxplot(data=features, x='churn_due_to_fraud', y='transaction_amount', palette='Set2')
plt.title('Transaction Amount Distribution by Churn Status', fontsize=16)
plt.xlabel('Churn Status', fontsize=14)
plt.ylabel('Transaction Amount', fontsize=14)
plt.xticks([0, 1], ['Not Churned', 'Churned'], rotation=0)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Violin plot for transaction amount by churn status
plt.figure(figsize=(10, 6))
sns.violinplot(data=features, x='churn_due_to_fraud', y='transaction_amount', palette='muted')
plt.title('Transaction Amount Distribution by Churn Status', fontsize=16)
plt.xlabel('Churn Status', fontsize=14)
plt.ylabel('Transaction Amount', fontsize=14)
plt.xticks([0, 1], ['Not Churned', 'Churned'], rotation=0)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap for correlation matrix
plt.figure(figsize=(12, 8))
correlation = features.corr()
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Feature Correlation Heatmap', fontsize=16)
plt.tight_layout()
plt.show()

#**Step 5: Prepare Data for Model Training**

In [ ]:
# Prepare features for modeling
X = features.drop(['customer_id', 'churn'], axis=1)
y = features['churn']

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Check the columns in the features DataFrame to find the correct churn column name
print("Features DataFrame Columns:", features.columns.tolist())

# Prepare features for modeling
# Replace 'churn' with the correct column name if it's different
X = features.drop(['customer_id', 'churn_due_to_fraud'], axis=1)  # Adjust 'churn_due_to_fraud' if needed
y = features['churn_due_to_fraud']  # Adjust 'churn_due_to_fraud' if needed

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shape of training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

#**Step 6: Train the XGBoost Model**

In [ ]:
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost model without use_label_encoder
model = XGBClassifier(eval_metric='logloss')

# Train the model
try:
    model.fit(X_train, y_train)
except AttributeError as e:
    print(f"Error during model fitting: {e}")

# Make predictions
try:
    y_pred = model.predict(X_test)
except ValueError as e:
    print(f"Error during prediction: {e}")

# Evaluate the model
try:
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:")
    # Handle UndefinedMetricWarning by using zero_division parameter
    print(classification_report(y_test, y_pred, zero_division=0))
except ValueError as e:
    print(f"Error during evaluation: {e}")

#**Step 7: Explainable AI using SHAP**

In [ ]:
import shap
import matplotlib.pyplot as plt

# Ensure you have the SHAP values for your test data
# Create the explainer
explainer = shap.Explainer(model)

# Calculate SHAP values for the test set
shap_values = explainer(X_test)

# Check the shape of SHAP values
print("SHAP values shape:", shap_values.shape)

# Create a summary plot for all features
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, X_test)
plt.title('SHAP Summary Plot for All Features', fontsize=16)
plt.show()

# Generate SHAP dependence plots for important features
for feature_name in X_test.columns:
    plt.figure(figsize=(12, 6))
    shap.dependence_plot(feature_name, shap_values.values, X_test)
    plt.title(f'SHAP Dependence Plot for {feature_name}', fontsize=16)
    plt.show()

In [ ]:
pip install --upgrade shap


In [ ]:
import shap
import matplotlib.pyplot as plt

# Ensure you have the SHAP values for your test data
# Create the explainer
explainer = shap.Explainer(model)

# Calculate SHAP values for the test set
shap_values = explainer(X_test)

# Print the shape of SHAP values
print("SHAP values shape:", shap_values.shape)

# Create a summary plot for all features
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test, show=False)  # Default color map is used
plt.title('SHAP Summary Plot for All Features', fontsize=16)
plt.tight_layout()
plt.show()

# Generate SHAP dependence plots for important features
important_features = X_test.columns[:5]  # Change this to the top N features based on your analysis
for feature_name in important_features:
    plt.figure(figsize=(12, 6))
    shap.dependence_plot(feature_name, shap_values.values, X_test, show=False)
    plt.title(f'SHAP Dependence Plot for {feature_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

# Additional Waterfall Plot for an individual instance
plt.figure(figsize=(12, 6))
shap.plots.waterfall(shap_values[0])  # Change index for different instances
plt.title('SHAP Waterfall Plot for First Instance', fontsize=16)
plt.tight_layout()
plt.show()

#**Step 8: Implement Retention Strategy Recommender**

In [ ]:
import pandas as pd

# Ensure that the index of X_test is reset to match with customer_id
X_test.reset_index(drop=True, inplace=True)

# Create a DataFrame with predictions and recommendations
predictions_df = pd.DataFrame({
    'customer_id': X_test.index,  # Use the index or a specific customer ID column if available
    'churn_probability': model.predict_proba(X_test)[:, 1]  # Get the probability of churn
})

# Apply the recommendation function to generate actions based on churn probability
predictions_df['recommendation'] = predictions_df['churn_probability'].apply(recommend_action)

# Output the predictions and recommendations
print(predictions_df[['customer_id', 'churn_probability', 'recommendation']])

#**Step 9: Save the Predictions for Submission**

In [ ]:
import os

# Define the directory to save the predictions (optional)
output_directory = 'output'  # You can specify any directory
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Define the path for the submission file
submission_file_path = os.path.join(output_directory, 'sample_submission.csv')

# Save the predictions for submission
predictions_df.to_csv(submission_file_path, index=False)

print(f"Predictions saved to {submission_file_path}")

In [ ]:
import pandas as pd

# Assuming you have a DataFrame named `pandas_df` with columns: 'customer_id', 'transaction_date', and 'transaction_amount'
# Make sure 'transaction_date' is in datetime format
pandas_df['date'] = pd.to_datetime(pandas_df['date'])

# Current date for recency calculation
current_date = pd.to_datetime('today')

# Calculate RFM
rfm_df = pandas_df.groupby('customer_id').agg({
    'date': lambda x: (current_date - x.max()).days,  # Recency
    'customer_id': 'count',  # Frequency (count of transactions)
    'transaction_amount': 'sum'  # Monetary (sum of transaction amounts)
}).reset_index()

# Rename the columns for clarity
rfm_df.columns = ['customer_id', 'recency', 'frequency', 'monetary']

# Display RFM DataFrame
print(rfm_df.head())

#**RFM**

In [ ]:
import pandas as pd

# Assuming you have a DataFrame named `pandas_df` with columns: 'customer_id', 'transaction_date', and 'transaction_amount'
# Make sure 'transaction_date' is in datetime format
pandas_df['date'] = pd.to_datetime(pandas_df['date'])

# Current date for recency calculation
current_date = pd.to_datetime('today')

# Calculate RFM
rfm_df = pandas_df.groupby('customer_id').agg(
    recency=('date', lambda x: (current_date - x.max()).days),  # Recency
    frequency=('transaction_date', 'count'),  # Frequency (count of transactions)
    monetary=('transaction_amount', 'sum')  # Monetary (sum of transaction amounts)
).reset_index()

# Display RFM DataFrame
print(rfm_df.head())

In [ ]:
duckdb.query('''
    CREATE OR REPLACE TABLE rfm_scores AS

    WITH rfm_feats AS (
        SELECT
            customer_id,
            date,
            DATEDIFF('days', MAX(date) OVER (), date) AS RECENCY,

            -- Aggregating transaction counts as FREQUENCY
            (
                atm_transfer_in_count +
                atm_transfer_out_count +
                bank_transfer_in_count +
                bank_transfer_out_count +
                crypto_in_count +
                crypto_out_count
            ) AS FREQUENCY,

            -- Summing monetary values as MONETARY
            (
                atm_transfer_in_volume +
                atm_transfer_out_volume +
                bank_transfer_in_volume +
                bank_transfer_out_volume +
                crypto_in_volume +
                crypto_out_volume
            ) AS MONETARY
        FROM train_running_stats
    ),
    percent_ranks AS (
        SELECT
            customer_id,
            date,
            PERCENT_RANK() OVER(PARTITION BY date ORDER BY RECENCY) AS RECENCY_PERCENTILE,
            PERCENT_RANK() OVER(PARTITION BY date ORDER BY FREQUENCY) AS FREQUENCY_PERCENTILE,
            PERCENT_RANK() OVER(PARTITION BY date ORDER BY MONETARY) AS MONETARY_PERCENTILE
        FROM rfm_feats
    ),
    rfm_feat_scores AS (
        SELECT
            customer_id,
            date,

            -- Assigning RECENCY scores
            CASE
                WHEN RECENCY_PERCENTILE <= 0.05 THEN 3
                WHEN RECENCY_PERCENTILE <= 0.50 THEN 2
                WHEN RECENCY_PERCENTILE <= 0.95 THEN 1
                ELSE 0
            END AS RECENCY_SCORE,

            -- Assigning FREQUENCY scores
            CASE
                WHEN FREQUENCY_PERCENTILE <= 0.05 THEN 0
                WHEN FREQUENCY_PERCENTILE <= 0.50 THEN 1
                WHEN FREQUENCY_PERCENTILE <= 0.95 THEN 2
                ELSE 3
            END AS FREQUENCY_SCORE,

            -- Assigning MONETARY scores
            CASE
                WHEN MONETARY_PERCENTILE <= 0.05 THEN 0
                WHEN MONETARY_PERCENTILE <= 0.50 THEN 1
                WHEN MONETARY_PERCENTILE <= 0.95 THEN 2
                ELSE 3
            END AS MONETARY_SCORE
        FROM percent_ranks
    )
    SELECT
        customer_id,
        date,
        RECENCY_SCORE,
        FREQUENCY_SCORE,
        MONETARY_SCORE,
        (
            RECENCY_SCORE +
            FREQUENCY_SCORE +
            MONETARY_SCORE
        ) AS TOTAL_RFM_SCORE
    FROM rfm_feat_scores
''')

In [ ]:
# Import necessary libraries
import duckdb

# Query to calculate RFM scores
query = '''
    CREATE OR REPLACE TABLE rfm_scores AS

    WITH base_features AS (
        SELECT
            customer_id,
            MAX(date) OVER () AS reference_date, -- Latest date in the dataset
            date,

            -- RECENCY: Days since the most recent transaction
            DATEDIFF('days', date, MAX(date) OVER (PARTITION BY customer_id)) AS RECENCY,

            -- FREQUENCY: Total number of transactions aggregated across all types
            (
                COALESCE(atm_transfer_in_count, 0) +
                COALESCE(atm_transfer_out_count, 0) +
                COALESCE(bank_transfer_in_count, 0) +
                COALESCE(bank_transfer_out_count, 0) +
                COALESCE(crypto_in_count, 0) +
                COALESCE(crypto_out_count, 0)
            ) AS FREQUENCY,

            -- MONETARY: Sum of transaction volumes across all types
            (
                COALESCE(atm_transfer_in_volume, 0) +
                COALESCE(atm_transfer_out_volume, 0) +
                COALESCE(bank_transfer_in_volume, 0) +
                COALESCE(bank_transfer_out_volume, 0) +
                COALESCE(crypto_in_volume, 0) +
                COALESCE(crypto_out_volume, 0)
            ) AS MONETARY
        FROM train_running_stats
    ),
    normalized_features AS (
        SELECT
            customer_id,
            RECENCY,
            FREQUENCY,
            MONETARY,

            -- Normalize each feature using min-max scaling for better scoring
            (RECENCY - MIN(RECENCY) OVER ()) / (MAX(RECENCY) OVER () - MIN(RECENCY) OVER ()) AS RECENCY_NORM,
            (FREQUENCY - MIN(FREQUENCY) OVER ()) / (MAX(FREQUENCY) OVER () - MIN(FREQUENCY) OVER ()) AS FREQUENCY_NORM,
            (MONETARY - MIN(MONETARY) OVER ()) / (MAX(MONETARY) OVER () - MIN(MONETARY) OVER ()) AS MONETARY_NORM
        FROM base_features
    ),
    rfm_scores_calculated AS (
        SELECT
            customer_id,

            -- Convert normalized values into categorical scores (3 = High, 0 = Low)
            CASE
                WHEN RECENCY_NORM <= 0.2 THEN 3
                WHEN RECENCY_NORM <= 0.5 THEN 2
                WHEN RECENCY_NORM <= 0.8 THEN 1
                ELSE 0
            END AS RECENCY_SCORE,

            CASE
                WHEN FREQUENCY_NORM <= 0.2 THEN 0
                WHEN FREQUENCY_NORM <= 0.5 THEN 1
                WHEN FREQUENCY_NORM <= 0.8 THEN 2
                ELSE 3
            END AS FREQUENCY_SCORE,

            CASE
                WHEN MONETARY_NORM <= 0.2 THEN 0
                WHEN MONETARY_NORM <= 0.5 THEN 1
                WHEN MONETARY_NORM <= 0.8 THEN 2
                ELSE 3
            END AS MONETARY_SCORE
        FROM normalized_features
    )
    SELECT
        customer_id,
        RECENCY_SCORE,
        FREQUENCY_SCORE,
        MONETARY_SCORE,
        -- Total RFM score
        (RECENCY_SCORE + FREQUENCY_SCORE + MONETARY_SCORE) AS TOTAL_RFM_SCORE
    FROM rfm_scores_calculated
'''

# Execute the query in DuckDB
duckdb.query(query)

# Fetch and display RFM scores as a DataFrame (optional)
rfm_scores = duckdb.query('SELECT * FROM rfm_scores').to_df()
print(rfm_scores.head())

In [ ]:
# Disable progress bar for cleaner output
duckdb.query('PRAGMA disable_progress_bar;')

# Base path to your Parquet files
BASEPATH = '/kaggle/input/neo-bank-non-sub-churn-prediction/'

# Paths for train and test data
train_path = BASEPATH + 'train_*.parquet'
test_path = BASEPATH + 'test.parquet'

# Load train data with advanced transformations
duckdb.query(f'''
    CREATE OR REPLACE VIEW train_data AS
    SELECT
        *,
        DATE_PART('year', date) AS year,   -- Extract year from date column
        DATE_PART('month', date) AS month, -- Extract month from date column
        CASE
            WHEN bank_transfer_in_volume + bank_transfer_out_volume > 10000 THEN 'High Value'
            WHEN bank_transfer_in_volume + bank_transfer_out_volume > 5000 THEN 'Medium Value'
            ELSE 'Low Value'
        END AS value_category, -- Categorize based on transaction volume
        REGEXP_EXTRACT(filename, r'train_(\\d{{4}})', 1) AS file_year -- Extract year from filename
    FROM read_parquet('{train_path}', union_by_name=True, filename=True)
''')

# Load test data with selective column exclusion and transformations
duckdb.query(f'''
    CREATE OR REPLACE VIEW test_data AS
    SELECT
        *,
        LENGTH(CAST(Usage AS VARCHAR)) AS usage_length, -- Compute length of Usage for advanced analysis
        IFNULL(atm_transfer_in_volume, 0) AS atm_transfer_in_filled, -- Fill missing values for ATM transfer
        CASE
            WHEN crypto_in_volume + crypto_out_volume > 0 THEN 'Active in Crypto'
            ELSE 'Inactive in Crypto'
        END AS crypto_activity -- Add crypto activity indicator
    FROM read_parquet('{test_path}', union_by_name=True, filename=True)
''')

# Verify train data schema
train_schema = duckdb.query('DESCRIBE train_data').df()
print("Train Data Schema:")
print(train_schema)

# Verify test data schema
test_schema = duckdb.query('DESCRIBE test_data').df()
print("\nTest Data Schema:")
print(test_schema)

# Example preview of data
train_preview = duckdb.query('SELECT * FROM train_data LIMIT 5').df()
test_preview = duckdb.query('SELECT * FROM test_data LIMIT 5').df()

print("\nTrain Data Preview:")
print(train_preview)

print("\nTest Data Preview:")
print(test_preview)

In [ ]:
import duckdb
import glob

# Path where Parquet files are uploaded in Colab
data_path = '/content/'

# Get the list of all Parquet files for training
train_files = glob.glob(data_path + 'train_*.parquet')
test_file = data_path + 'test.parquet'

# Initialize DuckDB connection
con = duckdb.connect()

# Create a view for train data by reading multiple Parquet files
con.execute(f"""
    CREATE OR REPLACE VIEW train_data AS
    SELECT
        *,
        ROW_NUMBER() OVER () AS row_id,  -- Unique identifier for debugging or sorting
        REPLACE(SPLIT_PART(filename, '/', -1), '.parquet', '') AS source_file -- File identifier
    FROM read_parquet({','.join([f"'{file}'" for file in train_files])},
                      union_by_name=True,
                      filename=True)
""")

# Create a view for test data!pwd
con.execute(f"""
    CREATE OR REPLACE VIEW test_data AS
    SELECT
        * EXCLUDE (Usage)  -- Modify column exclusion as needed
    FROM read_parquet('{test_file}', union_by_name=True, filename=True)
""")

# Display schema and sample rows for validation
print("Train Data Schema:\n")
print(con.execute("DESCRIBE train_data").fetchall())

print("\nTest Data Schema:\n")
print(con.execute("DESCRIBE test_data").fetchall())

# Optionally preview the data
print("\nTrain Data Sample:\n")
print(con.execute("SELECT * FROM train_data LIMIT 5").fetchdf())


print("\nTest Data Sample:\n")
print(con.execute("SELECT * FROM test_data LIMIT 5").fetchdf())

In [ ]:
!pwd

In [ ]:
import duckdb
import glob

# Path where Parquet files are uploaded in Colab
data_path = '/content/'

# Get all Parquet files for training using a dynamic wildcard
train_files_pattern = f"{data_path}train_*.parquet"
test_file = f"{data_path}test.parquet"

# Initialize DuckDB connection
con = duckdb.connect()

# Create a view for train data using the wildcard pattern
con.execute(f"""
    CREATE OR REPLACE VIEW train_data AS
    SELECT
        *,
        ROW_NUMBER() OVER () AS row_id,  -- Unique identifier for debugging or sorting
        REPLACE(SPLIT_PART(filename, '/', -1), '.parquet', '') AS source_file -- File identifier
    FROM read_parquet('{train_files_pattern}',
                      union_by_name=True,
                      filename=True)
""")

# Create a view for test data
con.execute(f"""
    CREATE OR REPLACE VIEW test_data AS
    SELECT
        * EXCLUDE (Usage)  -- Modify column exclusion as needed
    FROM read_parquet('{test_file}',
                      union_by_name=True,
                      filename=True)
""")

# Display schema and sample rows for validation
print("Train Data Schema:\n")
print(con.execute("DESCRIBE train_data").fetchall())

print("\nTest Data Schema:\n")
print(con.execute("DESCRIBE test_data").fetchall())

# Optionally preview the data
print("\nTrain Data Sample:\n")
print(con.execute("SELECT * FROM train_data LIMIT 5").fetchdf())

print("\nTest Data Sample:\n")
print(con.execute("SELECT * FROM test_data LIMIT 5").fetchdf())